# Anime_Score_Sim
Purpose, to generate anime that are similar to other anime by cosine similarity. Compare the  results of cosine similarity via score, via user scaled score, and via anime scaled score. 


In [1]:
import pandas as pd

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from scipy.sparse import csr_matrix

In [3]:
csvtoload = 'mal_scores_train_nonzero_v2.csv'# this is a subset of scores used for EDA and to design the system. Replace with a full csv, or to an sql database for the full dataset. 

In [4]:
ratings = pd.read_csv(csvtoload) # This is a subset of reviews with all 0's dropped. 
ratings['userid']=ratings['index']
ratings['score']=ratings['animescore']

In [5]:
ratings1 = ratings[ratings['score']!=0] # this is a sanity check to ensure no 0's got into the system. 

In [6]:
ratings1.head()

,Unnamed: 0,Unnamed: 0.1,animeid,animescore,index,name,status,unkey,userid,score,useravg,animeavg,score_anime_scaled,score_usr_scaled
0,0,13202,2001,10,Lancerevomr,Lancerevomr,COMPLETED,2001Lancerevomr,Lancerevomr,10,10.0,8.681996,1.318004,0.0
1,1,21990,10719,10,kuelpanda,kuelpanda,COMPLETED,10719kuelpanda,kuelpanda,10,9.5,7.535472,2.464528,0.5
2,2,21991,10155,9,kuelpanda,kuelpanda,COMPLETED,10155kuelpanda,kuelpanda,9,9.5,7.105895,1.894105,-0.5
3,3,22409,15051,10,vxxyman,vxxyman,COMPLETED,15051vxxyman,vxxyman,10,10.0,7.654129,2.345871,0.0
4,4,23792,1,10,pinheiro,pinheiro,COMPLETED,1pinheiro,pinheiro,10,10.0,8.804023,1.195977,0.0


In [7]:
ratings1.head()# checking our data

,Unnamed: 0,Unnamed: 0.1,animeid,animescore,index,name,status,unkey,userid,score,useravg,animeavg,score_anime_scaled,score_usr_scaled
0,0,13202,2001,10,Lancerevomr,Lancerevomr,COMPLETED,2001Lancerevomr,Lancerevomr,10,10.0,8.681996,1.318004,0.0
1,1,21990,10719,10,kuelpanda,kuelpanda,COMPLETED,10719kuelpanda,kuelpanda,10,9.5,7.535472,2.464528,0.5
2,2,21991,10155,9,kuelpanda,kuelpanda,COMPLETED,10155kuelpanda,kuelpanda,9,9.5,7.105895,1.894105,-0.5
3,3,22409,15051,10,vxxyman,vxxyman,COMPLETED,15051vxxyman,vxxyman,10,10.0,7.654129,2.345871,0.0
4,4,23792,1,10,pinheiro,pinheiro,COMPLETED,1pinheiro,pinheiro,10,10.0,8.804023,1.195977,0.0


In [8]:
#anime_pivot_table = ratings1.pivot_table(index='userid', columns = 'animeid',values='score_scaled')
# a pivot table  pandas function would be easier than what I do here but will run most systems out of memory. 

In [9]:
users = ratings.userid.unique()
animes = ratings.animeid.unique()

In [10]:
userid2idx = {o:i for i,o in enumerate(users)} # turn the user strings into a numerical value. 
animeid2idx = {o:i for i,o in enumerate(animes)} # remove missing anime numbers and re-order 

In [11]:
ratings1['anime_id'] = ratings1.animeid.apply(lambda x: animeid2idx[x]) #the above code set up mapping, this applies it. 
ratings1.userid = ratings1.userid.apply(lambda x: userid2idx[x])# the above code set up mapping, this applies it. 


We are using these mapping to generate which row and column each username and anime_id will go into. 

In [12]:


user_u = list(sorted(ratings1.animeid.unique()))
anime_id_u = list(sorted(ratings1.userid.unique()))

data = ratings1['score'].tolist() # Changing this line allows us to test score vs scaled score vs user scaled score
row = ratings1.userid.astype('category')
col = ratings1.anime_id.astype('category')
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(users), len(animes)))

In [13]:
reviewmatrix =sparse_matrix.T 

In [14]:
#col

In [15]:
#row

In [16]:
score_cos =cosine_similarity(reviewmatrix) # now we generatie our cosine of similarity

In [17]:
#animeinfo= pd.read_csv('animeinfo3.csv')
#animenames=pd.merge(ratings1,animeinfo,left_on='anime_id',right_on='animeid') # 

In [18]:
#ratings1.sort_values('anime_id')

In [19]:
inv_map = {v: k for k, v in  animeid2idx.items()} # we have to make an inverse map of the which row/column number is what anime number.


In [20]:
anime_names = pd.read_csv('Data/animeinfo3.csv').set_index('animeid')['name'].to_dict() # Then we map out anime_id to anime name

In [21]:
anime_names[32878] # confirming that our mapping is correct by going to https://myanimelist.net/anime/32878

'ēlDLIVE'

In [22]:
anime_in_order = []

In [23]:
for  anime in list(inv_map.values()): # we are going through the inverse map, and making a list of the anime names in the row/column order. 
    try:
        anime_in_order.append(anime_names[anime])
    except:
        anime_in_order.append('unknownanime'+str(anime)) # just in case an anime that I haven't gotten info on gets a review. 

In [24]:
anime_in_order

['Tengen Toppa Gurren Lagann',
 'Boku wa Tomodachi ga Sukunai',
 'Dog Days',
 'Love Live! School Idol Project',
 'Cowboy Bebop',
 'Highschool of the Dead',
 'Sword Art Online',
 'Chuunibyou demo Koi ga Shitai!',
 '07-Ghost',
 'Tokyo Ravens',
 'Natsume Yuujinchou San',
 'Crayon Shin-chan',
 'Shimoneta to Iu Gainen ga Sonzai Shinai Taikutsu na Sekai',
 'Clannad',
 'Aoi Bungaku Series',
 'Elfen Lied',
 'Hunter x Hunter (2011)',
 'Ansatsu Kyoushitsu (TV)',
 'Hajime no Ippo',
 'Kenpuu Denki Berserk',
 'Black★Rock Shooter',
 'Kiseijuu: Sei no Kakuritsu',
 'Gintama°',
 'Fullmetal Alchemist: Brotherhood',
 'Acchi Kocchi (TV)',
 'No Game No Life',
 'Gekkan Shoujo Nozaki-kun',
 'One Piece',
 'Nee Summer!',
 'One Punch Man',
 'Ueki no Housoku',
 'Can Can Bunny Extra',
 'Shoujo Kakumei Utena',
 'Doukyuusei (Movie)',
 'Lady Jewelpet',
 'Re:Zero kara Hajimeru Isekai Seikatsu',
 'Shingeki no Kyojin',
 'Tropical Kiss',
 'Akagami no Shirayuki-hime 2nd Season',
 'Clannad: After Story',
 'Steins;Gate',
 

In [25]:
rev1_sim_cos_df = pd.DataFrame(score_cos, index=anime_in_order , columns=anime_in_order)

In [70]:
rev1_sim_cos_df.head()

,Tengen Toppa Gurren Lagann,Boku wa Tomodachi ga Sukunai,Dog Days,Love Live! School Idol Project,Cowboy Bebop,Highschool of the Dead,Sword Art Online,Chuunibyou demo Koi ga Shitai!,07-Ghost,Tokyo Ravens,...,unknownanime29283,Ikenai Boy: Ikasu Maruhi Hand Power,unknownanime25623,Pikaia!!,unknownanime25597,unknownanime29361,Hello Kitty to Issho,unknownanime30468,unknownanime22493,Lovely Series
Tengen Toppa Gurren Lagann,1.000000,0.262517,0.149805,0.153384,0.406437,0.339543,0.361834,0.292844,0.117722,0.179530,...,0.002520,0.0,0.0,0.0,0.0,0.005670,0.0,0.0,0.0,0.0
Boku wa Tomodachi ga Sukunai,0.262517,1.000000,0.261582,0.221942,0.167863,0.378653,0.367601,0.450414,0.111146,0.308265,...,0.008814,0.0,0.0,0.0,0.0,0.005876,0.0,0.0,0.0,0.0
Dog Days,0.149805,0.261582,1.000000,0.138804,0.086168,0.201043,0.185011,0.215420,0.133285,0.227624,...,0.019353,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
Love Live! School Idol Project,0.153384,0.221942,0.138804,1.000000,0.099055,0.162902,0.212074,0.277650,0.056914,0.137558,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
Cowboy Bebop,0.406437,0.167863,0.086168,0.099055,1.000000,0.246013,0.259522,0.196765,0.093725,0.101847,...,0.006247,0.0,0.0,0.0,0.0,0.006247,0.0,0.0,0.0,0.0


In [26]:
rev1_sim_cos_df.to_csv('reviewsim.csv')

In [27]:
rev1_sim_cos_df.loc[:,'Girls und Panzer'].sort_values()[-20:]

Chuunibyou demo Koi ga Shitai!                         0.223767
So Ra No Wo To                                         0.225297
Strike Witches Movie                                   0.225497
Aoki Hagane no Arpeggio: Ars Nova                      0.229024
Non Non Biyori                                         0.231194
Stella Jogakuin Koutou-ka C³-bu                        0.232721
Kiniro Mosaic                                          0.232814
Upotte!!                                               0.234024
Yuru Yuri♪♪                                            0.235100
Girls und Panzer der Film Specials                     0.235398
Strike Witches 2                                       0.237462
Yuru Yuri                                              0.245536
Strike Witches                                         0.250606
High School Fleet                                      0.251997
Kantai Collection: KanColle                            0.269299
Girls und Panzer: Shoukai Shimasu!      

In [28]:
rev1_sim_cos_df.loc[:,'Death Note'].sort_values()[-20:]

Soul Eater                            0.416623
Highschool of the Dead                0.418852
Another                               0.425449
No Game No Life                       0.430089
Ao no Exorcist                        0.436494
Bleach                                0.438010
Angel Beats!                          0.445591
One Punch Man                         0.446467
Fullmetal Alchemist                   0.449778
Steins;Gate                           0.454060
Elfen Lied                            0.462743
Tokyo Ghoul                           0.469350
Mirai Nikki (TV)                      0.472048
Naruto                                0.484529
Code Geass: Hangyaku no Lelouch R2    0.488833
Sword Art Online                      0.501912
Fullmetal Alchemist: Brotherhood      0.508869
Code Geass: Hangyaku no Lelouch       0.522912
Shingeki no Kyojin                    0.559888
Death Note                            1.000000
Name: Death Note, dtype: float64

In [29]:
rev1_sim_cos_df.loc[:,'Bakemonogatari'].sort_values()[-20:]

Clannad                                                0.380303
Tengen Toppa Gurren Lagann                             0.380792
Fate/Zero                                              0.382524
Suzumiya Haruhi no Yuuutsu                             0.386893
Durarara!!                                             0.386992
Boku wa Tomodachi ga Sukunai                           0.387521
Mahou Shoujo Madoka★Magica                             0.392560
Chuunibyou demo Koi ga Shitai!                         0.410145
Ore no Imouto ga Konnani Kawaii Wake ga Nai            0.410161
Owarimonogatari                                        0.413966
Angel Beats!                                           0.418837
Yahari Ore no Seishun Love Comedy wa Machigatteiru.    0.420848
Steins;Gate                                            0.427492
Toradora!                                              0.434287
Tsukimonogatari                                        0.453740
Hanamonogatari                          

In [30]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]

Hataraku Maou-sama!                 0.444347
Tokyo Ghoul √A                      0.444828
High School DxD                     0.445173
Toradora!                           0.448540
Guilty Crown                        0.449455
Fullmetal Alchemist: Brotherhood    0.453053
Fairy Tail                          0.460510
One Punch Man                       0.463917
Highschool of the Dead              0.466987
Noragami                            0.470273
Ao no Exorcist                      0.488497
Akame ga Kill!                      0.496189
Death Note                          0.501912
Angel Beats!                        0.509218
Tokyo Ghoul                         0.515503
Mirai Nikki (TV)                    0.518953
No Game No Life                     0.547981
Shingeki no Kyojin                  0.593759
Sword Art Online II                 0.662262
Sword Art Online                    1.000000
Name: Sword Art Online, dtype: float64

In [31]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]

Hataraku Maou-sama!                 0.444347
Tokyo Ghoul √A                      0.444828
High School DxD                     0.445173
Toradora!                           0.448540
Guilty Crown                        0.449455
Fullmetal Alchemist: Brotherhood    0.453053
Fairy Tail                          0.460510
One Punch Man                       0.463917
Highschool of the Dead              0.466987
Noragami                            0.470273
Ao no Exorcist                      0.488497
Akame ga Kill!                      0.496189
Death Note                          0.501912
Angel Beats!                        0.509218
Tokyo Ghoul                         0.515503
Mirai Nikki (TV)                    0.518953
No Game No Life                     0.547981
Shingeki no Kyojin                  0.593759
Sword Art Online II                 0.662262
Sword Art Online                    1.000000
Name: Sword Art Online, dtype: float64

In [32]:


user_u = list(sorted(ratings1.animeid.unique()))
anime_id_u = list(sorted(ratings1.userid.unique()))

data = ratings1['score_anime_scaled'].tolist() # Changing this line allows us to test score vs scaled score vs user scaled score
row = ratings1.userid.astype('category')
col = ratings1.anime_id.astype('category')
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(users), len(animes)))

In [33]:
reviewmatrix =sparse_matrix.T 

In [34]:
#col

In [35]:
#row

In [36]:
score_cos =cosine_similarity(reviewmatrix) # now we generatie our cosine of similarity

In [37]:
#animeinfo= pd.read_csv('animeinfo3.csv')
#animenames=pd.merge(ratings1,animeinfo,left_on='anime_id',right_on='animeid') # 

In [38]:
#ratings1.sort_values('anime_id')

In [39]:
inv_map = {v: k for k, v in  animeid2idx.items()} # we have to make an inverse map of the which row/column number is what anime number.


In [40]:
anime_names = pd.read_csv('Data/animeinfo3.csv').set_index('animeid')['name'].to_dict() # Then we map out anime_id to anime name

In [41]:
anime_names[32878] # confirming that our mapping is correct by going to https://myanimelist.net/anime/32878

'ēlDLIVE'

In [42]:
anime_in_order = []

In [43]:
for  anime in list(inv_map.values()): # we are going through the inverse map, and making a list of the anime names in the row/column order. 
    try:
        anime_in_order.append(anime_names[anime])
    except:
        anime_in_order.append('unknownanime'+str(anime)) 

In [44]:
anime_in_order

['Tengen Toppa Gurren Lagann',
 'Boku wa Tomodachi ga Sukunai',
 'Dog Days',
 'Love Live! School Idol Project',
 'Cowboy Bebop',
 'Highschool of the Dead',
 'Sword Art Online',
 'Chuunibyou demo Koi ga Shitai!',
 '07-Ghost',
 'Tokyo Ravens',
 'Natsume Yuujinchou San',
 'Crayon Shin-chan',
 'Shimoneta to Iu Gainen ga Sonzai Shinai Taikutsu na Sekai',
 'Clannad',
 'Aoi Bungaku Series',
 'Elfen Lied',
 'Hunter x Hunter (2011)',
 'Ansatsu Kyoushitsu (TV)',
 'Hajime no Ippo',
 'Kenpuu Denki Berserk',
 'Black★Rock Shooter',
 'Kiseijuu: Sei no Kakuritsu',
 'Gintama°',
 'Fullmetal Alchemist: Brotherhood',
 'Acchi Kocchi (TV)',
 'No Game No Life',
 'Gekkan Shoujo Nozaki-kun',
 'One Piece',
 'Nee Summer!',
 'One Punch Man',
 'Ueki no Housoku',
 'Can Can Bunny Extra',
 'Shoujo Kakumei Utena',
 'Doukyuusei (Movie)',
 'Lady Jewelpet',
 'Re:Zero kara Hajimeru Isekai Seikatsu',
 'Shingeki no Kyojin',
 'Tropical Kiss',
 'Akagami no Shirayuki-hime 2nd Season',
 'Clannad: After Story',
 'Steins;Gate',
 

In [45]:
anime_rev_sim_cos_df = pd.DataFrame(score_cos, index=anime_in_order , columns=anime_in_order)
anime_rev_sim_cos_df.to_csv('anime_review_scaled.csv')

In [46]:
anime_rev_sim_cos_df.loc[:,'Girls und Panzer'].sort_values()[-20:]

Yuru Yuri                                              0.101225
Non Non Biyori                                         0.101438
Yama no Susume                                         0.102740
Aoki Hagane no Arpeggio: Ars Nova                      0.103374
Hanayamata                                             0.105395
Vividred Operation                                     0.105558
Yuru Yuri♪♪                                            0.108511
Strike Witches                                         0.113579
Girls und Panzer der Film Specials                     0.113582
Stella Jogakuin Koutou-ka C³-bu                        0.113762
Strike Witches 2                                       0.116868
Upotte!!                                               0.118917
Kiniro Mosaic                                          0.121561
Gochuumon wa Usagi desu ka?                            0.126501
Kantai Collection: KanColle                            0.127155
Girls und Panzer: Shoukai Shimasu!      

In [47]:
rev1_sim_cos_df.loc[:,'Death Note'].sort_values()[-20:]

Soul Eater                            0.416623
Highschool of the Dead                0.418852
Another                               0.425449
No Game No Life                       0.430089
Ao no Exorcist                        0.436494
Bleach                                0.438010
Angel Beats!                          0.445591
One Punch Man                         0.446467
Fullmetal Alchemist                   0.449778
Steins;Gate                           0.454060
Elfen Lied                            0.462743
Tokyo Ghoul                           0.469350
Mirai Nikki (TV)                      0.472048
Naruto                                0.484529
Code Geass: Hangyaku no Lelouch R2    0.488833
Sword Art Online                      0.501912
Fullmetal Alchemist: Brotherhood      0.508869
Code Geass: Hangyaku no Lelouch       0.522912
Shingeki no Kyojin                    0.559888
Death Note                            1.000000
Name: Death Note, dtype: float64

In [48]:
anime_rev_sim_cos_df.loc[:,'Bakemonogatari'].sort_values()[-20:]

Ookami to Koushinryou                                       0.112129
Mahou Shoujo Madoka★Magica                                  0.113089
Chuunibyou demo Koi ga Shitai!                              0.113803
Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.    0.115123
Katanagatari                                                0.115370
Kizumonogatari II: Nekketsu-hen                             0.116033
Durarara!!                                                  0.118250
Toradora!                                                   0.121156
Suzumiya Haruhi no Yuuutsu                                  0.121681
Hyouka                                                      0.122309
Steins;Gate                                                 0.129357
Kizumonogatari I: Tekketsu-hen                              0.131787
Yahari Ore no Seishun Love Comedy wa Machigatteiru.         0.132268
Owarimonogatari                                             0.184879
Hanamonogatari                    

In [49]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]

Hataraku Maou-sama!                 0.444347
Tokyo Ghoul √A                      0.444828
High School DxD                     0.445173
Toradora!                           0.448540
Guilty Crown                        0.449455
Fullmetal Alchemist: Brotherhood    0.453053
Fairy Tail                          0.460510
One Punch Man                       0.463917
Highschool of the Dead              0.466987
Noragami                            0.470273
Ao no Exorcist                      0.488497
Akame ga Kill!                      0.496189
Death Note                          0.501912
Angel Beats!                        0.509218
Tokyo Ghoul                         0.515503
Mirai Nikki (TV)                    0.518953
No Game No Life                     0.547981
Shingeki no Kyojin                  0.593759
Sword Art Online II                 0.662262
Sword Art Online                    1.000000
Name: Sword Art Online, dtype: float64

In [50]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]

Hataraku Maou-sama!                 0.444347
Tokyo Ghoul √A                      0.444828
High School DxD                     0.445173
Toradora!                           0.448540
Guilty Crown                        0.449455
Fullmetal Alchemist: Brotherhood    0.453053
Fairy Tail                          0.460510
One Punch Man                       0.463917
Highschool of the Dead              0.466987
Noragami                            0.470273
Ao no Exorcist                      0.488497
Akame ga Kill!                      0.496189
Death Note                          0.501912
Angel Beats!                        0.509218
Tokyo Ghoul                         0.515503
Mirai Nikki (TV)                    0.518953
No Game No Life                     0.547981
Shingeki no Kyojin                  0.593759
Sword Art Online II                 0.662262
Sword Art Online                    1.000000
Name: Sword Art Online, dtype: float64

In [51]:


user_u = list(sorted(ratings1.animeid.unique()))
anime_id_u = list(sorted(ratings1.userid.unique()))

data = ratings1['score_usr_scaled'].tolist() # Changing this line allows us to test score vs scaled score vs user scaled score
row = ratings1.userid.astype('category')
col = ratings1.anime_id.astype('category')
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(users), len(animes)))

In [52]:
reviewmatrix =sparse_matrix.T 

In [53]:
#col

In [54]:
#row

In [55]:
score_cos =cosine_similarity(reviewmatrix) # now we generatie our cosine of similarity

In [56]:
#animeinfo= pd.read_csv('animeinfo3.csv')
#animenames=pd.merge(ratings1,animeinfo,left_on='anime_id',right_on='animeid') # 

In [57]:
#ratings1.sort_values('anime_id')

In [58]:
inv_map = {v: k for k, v in  animeid2idx.items()} # we have to make an inverse map of the which row/column number is what anime number.


In [59]:
anime_names = pd.read_csv('Data/animeinfo3.csv').set_index('animeid')['name'].to_dict() # Then we map out anime_id to anime name

In [60]:
anime_names[32878] # confirming that our mapping is correct by going to https://myanimelist.net/anime/32878

'ēlDLIVE'

In [61]:
anime_in_order = []

In [62]:
for  anime in list(inv_map.values()): # we are going through the inverse map, and making a list of the anime names in the row/column order. 
    try:
        anime_in_order.append(anime_names[anime])
    except:
        anime_in_order.append('unknownanime'+str(anime)) 

In [63]:
anime_in_order

['Tengen Toppa Gurren Lagann',
 'Boku wa Tomodachi ga Sukunai',
 'Dog Days',
 'Love Live! School Idol Project',
 'Cowboy Bebop',
 'Highschool of the Dead',
 'Sword Art Online',
 'Chuunibyou demo Koi ga Shitai!',
 '07-Ghost',
 'Tokyo Ravens',
 'Natsume Yuujinchou San',
 'Crayon Shin-chan',
 'Shimoneta to Iu Gainen ga Sonzai Shinai Taikutsu na Sekai',
 'Clannad',
 'Aoi Bungaku Series',
 'Elfen Lied',
 'Hunter x Hunter (2011)',
 'Ansatsu Kyoushitsu (TV)',
 'Hajime no Ippo',
 'Kenpuu Denki Berserk',
 'Black★Rock Shooter',
 'Kiseijuu: Sei no Kakuritsu',
 'Gintama°',
 'Fullmetal Alchemist: Brotherhood',
 'Acchi Kocchi (TV)',
 'No Game No Life',
 'Gekkan Shoujo Nozaki-kun',
 'One Piece',
 'Nee Summer!',
 'One Punch Man',
 'Ueki no Housoku',
 'Can Can Bunny Extra',
 'Shoujo Kakumei Utena',
 'Doukyuusei (Movie)',
 'Lady Jewelpet',
 'Re:Zero kara Hajimeru Isekai Seikatsu',
 'Shingeki no Kyojin',
 'Tropical Kiss',
 'Akagami no Shirayuki-hime 2nd Season',
 'Clannad: After Story',
 'Steins;Gate',
 

In [64]:

user_rev_sim_cos_df = pd.DataFrame(score_cos, index=anime_in_order , columns=anime_in_order)
user_rev_sim_cos_df.to_csv('user_review_scaled.csv')

In [65]:
user_rev_sim_cos_df.loc[:,'Girls und Panzer'].sort_values()[-20:]

Yuru Yuri                                              0.048409
Kiniro Mosaic                                          0.048513
Kappa Kawatarou                                        0.048732
Jiaoao de Jiangjun                                     0.048732
Girls und Panzer Heartful Tank Disc Picture Drama      0.050416
Yuru Yuri Nachuyachumi!+                               0.052179
Non Non Biyori                                         0.052979
unknownanime31054                                      0.053040
Gochuumon wa Usagi desu ka?                            0.053517
Hidamari Sketch x ☆☆☆                                  0.054318
Mahou Shoujo Lyrical Nanoha: The Movie 2nd A's         0.054592
Strike Witches Movie                                   0.057094
Shirobako                                              0.060092
Gochuumon wa Usagi desu ka??                           0.061483
Yuru Yuri♪♪                                            0.062068
Girls und Panzer der Film Specials      

In [66]:
rev1_sim_cos_df.loc[:,'Death Note'].sort_values()[-20:]

Soul Eater                            0.416623
Highschool of the Dead                0.418852
Another                               0.425449
No Game No Life                       0.430089
Ao no Exorcist                        0.436494
Bleach                                0.438010
Angel Beats!                          0.445591
One Punch Man                         0.446467
Fullmetal Alchemist                   0.449778
Steins;Gate                           0.454060
Elfen Lied                            0.462743
Tokyo Ghoul                           0.469350
Mirai Nikki (TV)                      0.472048
Naruto                                0.484529
Code Geass: Hangyaku no Lelouch R2    0.488833
Sword Art Online                      0.501912
Fullmetal Alchemist: Brotherhood      0.508869
Code Geass: Hangyaku no Lelouch       0.522912
Shingeki no Kyojin                    0.559888
Death Note                            1.000000
Name: Death Note, dtype: float64

In [67]:
user_rev_sim_cos_df.loc[:,'Bakemonogatari'].sort_values()[-20:]

Clannad: After Story                0.167413
Ookami to Koushinryou               0.168238
Kara no Kyoukai 5: Mujun Rasen      0.175337
Tengen Toppa Gurren Lagann          0.181864
Fate/Zero                           0.183347
Mahou Shoujo Madoka★Magica          0.189223
Kizumonogatari III: Reiketsu-hen    0.189781
Fate/Zero 2nd Season                0.196545
Katanagatari                        0.204871
Suzumiya Haruhi no Shoushitsu       0.230126
Steins;Gate                         0.238379
Kizumonogatari II: Nekketsu-hen     0.241683
Kizumonogatari I: Tekketsu-hen      0.254262
Hanamonogatari                      0.305775
Tsukimonogatari                     0.314627
Owarimonogatari                     0.324273
Nekomonogatari: Kuro                0.370102
Nisemonogatari                      0.453346
Monogatari Series: Second Season    0.470590
Bakemonogatari                      1.000000
Name: Bakemonogatari, dtype: float64

In [68]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]

Hataraku Maou-sama!                 0.444347
Tokyo Ghoul √A                      0.444828
High School DxD                     0.445173
Toradora!                           0.448540
Guilty Crown                        0.449455
Fullmetal Alchemist: Brotherhood    0.453053
Fairy Tail                          0.460510
One Punch Man                       0.463917
Highschool of the Dead              0.466987
Noragami                            0.470273
Ao no Exorcist                      0.488497
Akame ga Kill!                      0.496189
Death Note                          0.501912
Angel Beats!                        0.509218
Tokyo Ghoul                         0.515503
Mirai Nikki (TV)                    0.518953
No Game No Life                     0.547981
Shingeki no Kyojin                  0.593759
Sword Art Online II                 0.662262
Sword Art Online                    1.000000
Name: Sword Art Online, dtype: float64

In [69]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]

Hataraku Maou-sama!                 0.444347
Tokyo Ghoul √A                      0.444828
High School DxD                     0.445173
Toradora!                           0.448540
Guilty Crown                        0.449455
Fullmetal Alchemist: Brotherhood    0.453053
Fairy Tail                          0.460510
One Punch Man                       0.463917
Highschool of the Dead              0.466987
Noragami                            0.470273
Ao no Exorcist                      0.488497
Akame ga Kill!                      0.496189
Death Note                          0.501912
Angel Beats!                        0.509218
Tokyo Ghoul                         0.515503
Mirai Nikki (TV)                    0.518953
No Game No Life                     0.547981
Shingeki no Kyojin                  0.593759
Sword Art Online II                 0.662262
Sword Art Online                    1.000000
Name: Sword Art Online, dtype: float64